# Variables and setting up a session 

In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

f = x*x*y + y + 2


c:\users\yc\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Setting up a session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)
print(result)
sess.close()    #always be closing

42


In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)    

42


In [4]:
#Using tf.global_variables_initializer() to initialize all variables together
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)    #or init.run()
    result = f.eval()
print(result)

42


In [5]:
#With InteractiveSession()
sess = tf.InteractiveSession()
init.run()    #or sess.run(init)
result = f.eval()
#print(result)

#you have to close the session if you use InteractiveSession()
sess.close()

# Managing Graphs

In [6]:
#Any node you create is automatically added to the default graph
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()


True

In [7]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph)    #it is a graph
print(x2.graph is tf.get_default_graph)    #but not the  default graph
tf.reset_default_graph()

True
False


In [8]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x *3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


# Linear Regression with Tensorflow

In [12]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]    #added a column vector of m 1s

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

#In the evaluation phase, we will have to evaluate theta
with tf.Session() as sess:
    theta_value = theta.eval()

#This was the normal equation method
print(housing)    #what the data looks like

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]]), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]), 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/da

# Manually computing Gradients for GD
#### Batch Gradient Descent

In [22]:
from sklearn.preprocessing import StandardScaler
n_epochs = 1000    #times it will do forward and backward passes
learning_rate = 0.01

#We need to scale the Data for Gradient Descent
scaler = StandardScaler().fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

#Construction phase
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients


#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)    #we construct the best theta here, eval later to find best
    best_theta = theta.eval()
print("Best theta: ", best_theta)

Epoch:  0 MSE =  8.065985
Epoch:  100 MSE =  5.0069027
Epoch:  200 MSE =  4.9445987
Epoch:  300 MSE =  4.90945
Epoch:  400 MSE =  4.883471
Epoch:  500 MSE =  4.864137
Epoch:  600 MSE =  4.849684
Epoch:  700 MSE =  4.838844
Epoch:  800 MSE =  4.8306704
Epoch:  900 MSE =  4.824484
Best theta:  [[-0.27591395]
 [ 0.8861117 ]
 [ 0.16327482]
 [-0.30879635]
 [ 0.31356624]
 [ 0.01096052]
 [-0.04429711]
 [-0.52227587]
 [-0.49643603]]


## Using an Optimizer

In [23]:
#Add this in place of the gradients = [...] line

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = optimizer.minimize(mse)


In [24]:
#to feed data to algorithm, we make some modifications to the code:

#Construction phase
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch 

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients


#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})    #placeholders are fed with this dict
        best_theta = theta.eval()
            
print("Best theta: ", best_theta)


Best theta:  [[-0.27219152]
 [ 0.8377844 ]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]


# Saving and restoring models

- Create a Saver node at the end of construction phase

In [26]:
#Construction phase
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients

#Saver:
saver = tf.train.Saver()

#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)    #we construct the best theta here, eval later to find best
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/ycModel.ckpt")
print("Best theta: ", best_theta)


Epoch:  0 MSE =  8.813791
Epoch:  100 MSE =  5.139094
Epoch:  200 MSE =  4.9726214
Epoch:  300 MSE =  4.922286
Epoch:  400 MSE =  4.889352
Epoch:  500 MSE =  4.8657355
Epoch:  600 MSE =  4.8486533
Epoch:  700 MSE =  4.836275
Epoch:  800 MSE =  4.827299
Epoch:  900 MSE =  4.8207903
Best theta:  [[-0.22573972]
 [ 0.7267378 ]
 [ 0.14211889]
 [ 0.00863735]
 [ 0.04389765]
 [ 0.00533218]
 [-0.0391091 ]
 [-0.82614493]
 [-0.7809662 ]]


### To  restore

In [ ]:
#Instead of using the init node during execution,
with tf.Session() as sess:
    saver.restore(sess, "tmp/ycModel.ckpt")


# Visualizing with TensorBoard

- We need to use a different log directory everytime we run our program, else tf merges stats from two runs

In [33]:
tf.reset_default_graph()    #Clears the default graph stack and resets the global default graph.

from datetime import datetime  

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)


In [35]:
#Construction phase
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")    #tensor containing random values, given shape and range
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)    #-> Normal way
gradients = tf.gradients(mse, [theta])[0]              #-> Tensorflow way (efficient and accurate)
training_op = tf.assign(theta, theta - learning_rate * gradients)    #theta = theta - lr*gradients

#Adding at end of construction for tensorboard:
mse_summary = tf.summary.scalar('MSE', mse)    #binary log string
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


In [36]:
#Execution Phase
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            #Adding for tensorboard:
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
        
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})    #placeholders are fed with this dict
        best_theta = theta.eval()
            
print("Best theta: ", best_theta)
#close FileWriter at the end of program
file_writer.close()

Best theta:  [[ 0.6303098 ]
 [ 0.8377844 ]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]
